In [1]:
from datasets import Dataset

c:\Users\ecoskun\Desktop\BachelorarbeitAbgabe\bachelorarbeit_eyuep_coskun_3594788\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Vorbereitung

### Data Load

In [2]:
import json
try:
    with open('../LLM_Austausch_Experiment/wizard_antworten.json', 'r') as datei:
        json_data = json.load(datei)
except Exception as errror:
    print(f'errror: {errror}')
    
dataset=json_data['wizard_ergebnisse']

### RAGAS Metriks bestimmen

In [3]:
from ragas import evaluate
from ragas.metrics import (
   answer_similarity
)
metrics = [
   answer_similarity
]

### Embeddings Model Ändern

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
model_deutsche_telekom = HuggingFaceEmbeddings(model_name="deutsche-telekom/gbert-large-paraphrase-cosine")

# RAG Evaluation

### RAG-Ergebnis von Json-Format zu Hugging Face Dataset

In [6]:
RAG_data_samples={
'question':[],
'answer':[],
'ground_truth':[]
}
for data_point in dataset:
    RAG_data_samples['question'].append(data_point['frage'])
    RAG_data_samples['answer'].append(data_point['RAG_wizard_antwort'])
    RAG_data_samples['ground_truth'].append(data_point['ref_antwort'])
   
RAG_dataset = Dataset.from_dict(RAG_data_samples)
RAG_dataset

Dataset({
    features: ['question', 'answer', 'ground_truth'],
    num_rows: 48
})

### Evaluation von RAG

In [7]:
RAG_results = evaluate(RAG_dataset, metrics,raise_exceptions=False,embeddings=model_deutsche_telekom)
RAG_results

Evaluating: 100%|██████████| 48/48 [00:14<00:00,  3.38it/s]


{'answer_similarity': 0.6778}

In [8]:
RAG_df=RAG_results.to_pandas()
sort_RAG_ergebnis=RAG_df.sort_values(by='answer_similarity')
sort_RAG_ergebnis

,question,answer,ground_truth,answer_similarity
43,Wo befinden sich die PartnerwerkstÃ¤tten?,Es gibt keine spezifische Information Ã¼ber di...,Die VHV hat ein bundesweites Netz zertifiziert...,0.247902
24,Ich habe meinen Antrag widerrufen – bekomme ic...,Ich habe meinen Antrag widerrufen - bekomme ic...,"Ja, Sie erhalten eine WiderrufsbestÃ¤tigung. S...",0.381796
14,"Kann ich meinen Vertrag in der Vergangenheit, ...","Nein, Sie kÃ¶nnen Ihren Vertrag nicht durch RÃ...","Nein, das geht leider nicht. Eine RÃ¼ckdatier...",0.385032
3,Was bietet mir die Zusatzleistung Schutzbrief?,"\nDie Zusatzleistung ""Schutzbrief"" ist Teil Ih...","Egal, ob Autopanne oder Unfall, Diebstahl oder...",0.427548
4,Welche Versicherungen fÃ¼r Zweitwagen oder and...,Es werden folgende Versicherungsarten fÃ¼r Zwe...,"Egal, ob Zweitwagen-, Ehegatten- oder FahranfÃ...",0.457992
44,Wie kann man eine RÃ¼ckstufung nach einem Unfa...,Zur Vermeidung einer RÃ¼ckstufung nach einem U...,"Ein Schaden, fÃ¼r den die VHV eine EntschÃ¤dig...",0.489462
21,KÃ¶nnen Sie meinen bestehenden Vertrag beim Vo...,"Ja, Sie kÃ¶nnen Ihren bestehenden Vertrag beim...","Nein, das darf die VHV aus wettbewerbsrechtlic...",0.496151
32,Was bedeutet die Mallorca-Police?,"Die ""Mallorca-Policy"" ist ein Sprachspiel und ...",Als Mallorca-Police bezeichnet man die Mitvers...,0.510148
28,Wie kann ich meinen Vertrag kÃ¼ndigen?,\nSie kÃ¶nnen Ihren Versicherungsvertrag in zw...,Sie kÃ¶nnen Ihr KÃ¼ndigungsschreiben unter Ang...,0.518217
42,Was muss bei einem Glasschaden beachtet werden?,"Bei einem Glasschaden ist es wichtig, die Sich...",Es wird auf den Abzug der Selbstbeteiligung ve...,0.537921


# Pure-LLM Evaluation

### Pure-LLM-antworten von Json-Format zu Hugging Face Dataset

In [9]:
base_data_samples={
'question':[],
'answer':[],
'ground_truth':[]
}
for data_point in dataset:
    base_data_samples['question'].append(data_point['frage'])
    base_data_samples['answer'].append(data_point['Pure_LLM_wizard_antwort'])
    base_data_samples['ground_truth'].append(data_point['ref_antwort'])
   
base_dataset = Dataset.from_dict(base_data_samples)
base_dataset


Dataset({
    features: ['question', 'answer', 'ground_truth'],
    num_rows: 48
})

### Evaluation von Pure-LLM-Antworten 

In [10]:
base_llm_results = evaluate(base_dataset, metrics,raise_exceptions=False,embeddings=model_deutsche_telekom)
base_llm_results

Evaluating:   0%|          | 0/48 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 48/48 [00:17<00:00,  2.77it/s]


{'answer_similarity': 0.5948}

In [11]:
base_df=base_llm_results.to_pandas()
sort_base_llm_ergebnis=base_df.sort_values(by='answer_similarity')

sort_base_llm_ergebnis


,question,answer,ground_truth,answer_similarity
43,Wo befinden sich die PartnerwerkstÃ¤tten?,Die genauen Standorten der PartnerwerkstÃ¤tte...,Die VHV hat ein bundesweites Netz zertifiziert...,0.263145
32,Was bedeutet die Mallorca-Police?,"Die ""Mallorca Police"" ist eine BehÃ¶rde, die ...",Als Mallorca-Police bezeichnet man die Mitvers...,0.324589
8,Kann ich einen abweichenden Halter versichern?,"Ja, Sie kÃ¶nnen Ihren Hund oder Ihr Haustier ...","Ja, das ist mÃ¶glich. FÃ¼r die Antragstellung ...",0.332188
24,Ich habe meinen Antrag widerrufen – bekomme ic...,"Ja, Sie sollten eine BestÃ¤tigung erhalten, w...","Ja, Sie erhalten eine WiderrufsbestÃ¤tigung. S...",0.349899
36,Wann erfolgt eine VerÃ¤nderung der Schadenfrei...,Die VerÃ¤nderungen in den Schadenfreiheitskla...,"Wer Jahr fÃ¼r Jahr unfallfrei fÃ¤hrt, profitie...",0.363800
44,Wie kann man eine RÃ¼ckstufung nach einem Unfa...,Um eine RÃ¼ckstufung nach einem Unfall zu ver...,"Ein Schaden, fÃ¼r den die VHV eine EntschÃ¤dig...",0.377119
28,Wie kann ich meinen Vertrag kÃ¼ndigen?,"Um einen Vertrag zu kÃ¼ndigen, mÃ¼ssen Sie no...",Sie kÃ¶nnen Ihr KÃ¼ndigungsschreiben unter Ang...,0.409168
46,Ich habe einen Schutzbrief abgeschlossen - wie...,Wenn Sie einen Schutzbrief abgeschlossen habe...,Bei einer Panne mit Ihrem Fahrzeug setzen Sie ...,0.429595
27,Ich habe mein Fahrzeug verkauft – wie beende i...,"Um einen Vertrag zu beenden, mÃ¼ssen Sie in d...",FÃ¤llt das versicherte Risiko weg (wegen Autov...,0.437047
31,Was bedeutet Werkstattbindung?,"""Werkstattbindung"" ist ein Begriff, der in De...",Mit der Werkstattbindung sichern Sie sich scho...,0.455670
